In [ ]:
# Import packages
import pandas as pd # Data manipulation
import numpy as np # Numeric computation
import matplotlib.pyplot as plt # Visualisation
import seaborn as sns # Visualisation

# Prepare Data
## Import 

In [ ]:
# Import dataset

## Explore

In [ ]:
# Dimensions of the dataset


In [ ]:
# Dataset structure


In [ ]:
# View first five rows


In [ ]:
# Describe dataset


In [ ]:
# Distributions

## Split

In [ ]:
# Train-test split


# Build model
## Baseline

## Iterate

# Evaluate

# Communicate